# The nPrint Machine Learning Pipeline

This notebook is designed to give you a **very** simple example of how to use nPrint in a generic machine learning pipeline and the rapid pace at which we can train new models and test new ideas on network traffic. Note that this example is simply to show the pipeline, not to test a hard problem. The traffic collected is to the same website over the course of about 15 seconds.


### Requirements

nPrint must be installed into $PATH for external commands to work

### Directory Sturcture

There are 2 `pcap` files in this directory
1. `port443.pcap` - a small trace of packets sent and received over https  
2. `port80.pcap` - a small trace of packets sent and received over http

# Generating nPrints from Traffic

First, lets generate nPrints from each traffic trace. Let's **only** include the TCP headers in the nPrints for now.

Lets examine the nPrints, which can be directly loaded with Pandas

In [ ]:
import pandas as pd

friday = pd.read_csv('TrafficLabelling/Friday-WorkingHours-Morning.pcap_ISCX.csv.gz', index_col=0)

print('Friday nPrint: Number of Packets: {0}, Features per packet: {1}'.format(friday.shape[0], friday.shape[1]))

In [ ]:
friday

## nPrint to Machine Learning Samples

We label each sample with its corresponding operating system by matching its source IP address to the OS found
on this website https://www.unb.ca/cic/datasets/ids-2017.html

In [ ]:
import numpy as np
import math 
samples = []
labels = []

friday = friday.replace([np.inf, -np.inf], np.nan)
friday = friday.dropna()
# label the operating system based on IP address from this website: https://www.unb.ca/cic/datasets/ids-2017.html
ip_to_os = {'205.174.165.73': 'Kali', '205.174.165.69': 'Win', '205.174.165.70': 'Win', '205.174.165.71': 'Win', '192.168.10.50': 'Web server 16 Public', '192.168.10.205.174.165.68': 'Web server 16 Public', '192.168.10.51': 'Ubuntu server 12 Public', '192.168.10.205.174.165.66': 'Ubuntu server 12 Public', '192.168.10.19': 'Ubuntu 14.4, 32B', '192.168.10.17': 'Ubuntu 14.4, 64B', '192.168.10.16': 'Ubuntu 16.4, 32B', '192.168.10.12': 'Ubuntu 16.4, 64B', '192.168.10.9': 'Win 7 Pro, 64B', '192.168.10.5': 'Win 8.1, 64B', '192.168.10.8': 'Win Vista, 64B', '192.168.10.14': 'Win 10, pro 32B', '192.168.10.15': 'Win 10, 64B', '192.168.10.25': 'MAC'}
labels = friday[' Source IP'].apply(lambda x : "Other" if x not in ip_to_os else ip_to_os[x])
samples = friday.drop([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Timestamp'], axis=1)
samples = samples._get_numeric_data()
samples = samples.reset_index(drop=True)

samples = samples.astype(np.float32)
samples.dtypes.value_counts()

## Training a Classifier

We're already ready to train and test a model on the traffic we gathered. Let's split the data into training and testing data, train a model, and get a stat report.

### Random Forest Classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(samples, labels)

# Initialize Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)

# Train 
clf.fit(X_train, y_train) 

# Predict
y_pred = clf.predict(X_test)

# Statistics

# First, lets get a stat report about the precision and recall:
report = classification_report(y_test, y_pred)
print(report)

# Let's also get the ROC AUC score while we're here, which requires a probability instead of just the prediction
y_pred_proba = clf.predict_proba(X_test)
# predict_proba gives us a probability estimate of each class, while roc_auc just cares about the "positive" class
y_pred_proba_pos = [sublist[1] for sublist in y_pred_proba]

In [ ]:
roc = roc_auc_score(y_test, y_pred_proba, multi_class="ovo")
print('ROC AUC Score: {0}'.format(roc))

### ExtraTrees Classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# Split data
X_train, X_test, y_train, y_test = train_test_split(samples, labels)

# Initialize Classifier
clf = ExtraTreesClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)

# Train 
clf.fit(X_train, y_train) 

# Predict
y_pred = clf.predict(X_test)

# Statistics

# First, lets get a stat report about the precision and recall:
report = classification_report(y_test, y_pred)
print(report)

# Let's also get the ROC AUC score while we're here, which requires a probability instead of just the prediction
y_pred_proba = clf.predict_proba(X_test)
# predict_proba gives us a probability estimate of each class, while roc_auc just cares about the "positive" class
y_pred_proba_pos = [sublist[1] for sublist in y_pred_proba]

In [ ]:
roc = roc_auc_score(y_test, y_pred_proba, multi_class="ovo")
print('ROC AUC Score: {0}'.format(roc))

## Understanding the model

nPrint's alignment of each packet allows for understanding the specific features (parts of the packet) that are driving the model's performance. It turns out that the options that are being set in the TCP header is actually more important than the port numbers themselves!

In [ ]:
# Get Raw feature importances
feature_importances = clf.feature_importances_
# Match the feature names we know with the importances
named_importances = []
for column_name, importance in zip(nprint_80.columns, feature_importances):
    named_importances.append((column_name, importance))
# Sort the named feature importances
sorted_feature_importances = sorted(named_importances, key=lambda tup: tup[1], reverse=True)
# Now lets print the top 20 important features (bits)
print(*sorted_feature_importances[0:20], sep='\n') 

## Rapidly testing different versions of the problem

now that we have a generic pipeline, we can leverage nPrint's flags to generate different versions of nPrints. Let's test a version of this classification problem using **only** the IPv4 Headers of the packets

In [ ]:
# Generate nPrints
cmd_80 = 'nprint -P port80.pcap -4  -W port80.npt'
cmd_443 = 'nprint -P port443.pcap -4 -W port443.npt'
!{cmd_80}
!{cmd_443}

# Load nPrints
nprint_80 = pd.read_csv('port80.npt', index_col=0)
nprint_443 = pd.read_csv('port443.npt', index_col=0)

# Assoicate with Labels
samples = []
labels = []
for _, row in nprint_80.iterrows():
    samples.append(np.array(row))
    labels.append('unencrypted')

for _, row in nprint_443.iterrows():
    samples.append(np.array(row))
    labels.append('encrypted')
    
# Train and Test the Classifier
# Split data
X_train, X_test, y_train, y_test = train_test_split(samples, labels)
# Initialize Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)
# Train 
clf.fit(X_train, y_train) 
# Predict
y_pred = clf.predict(X_test)
# Statistics
report = classification_report(y_test, y_pred)
print(report)



How about Testing using just the first 30 payload bytes in each packet?

In [ ]:
# Generate nPrints
cmd_80 = 'nprint -P port80.pcap -p 30 -W port80.npt'
cmd_443 = 'nprint -P port443.pcap -p 30 -W port443.npt'
!{cmd_80}
!{cmd_443}

# Load nPrints
nprint_80 = pd.read_csv('port80.npt', index_col=0)
nprint_443 = pd.read_csv('port443.npt', index_col=0)

# Assoicate with Labels
samples = []
labels = []
for _, row in nprint_80.iterrows():
    samples.append(np.array(row))
    labels.append('unencrypted')

for _, row in nprint_443.iterrows():
    samples.append(np.array(row))
    labels.append('encrypted')
    
# Train and Test the Classifier
# Split data
X_train, X_test, y_train, y_test = train_test_split(samples, labels)
# Initialize Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)
# Train 
clf.fit(X_train, y_train) 
# Predict
y_pred = clf.predict(X_test)
# Statistics
report = classification_report(y_test, y_pred)
print(report)

A much harder problem, with a much lower score. It may be likely that many packets don't have a payload at all, making it impossible to guess the traffic! What if we remove those packets from our dataset?

In [9]:
# Load nPrints
nprint_80 = pd.read_csv('port80.npt', index_col=0)
nprint_443 = pd.read_csv('port443.npt', index_col=0)

# Assoicate with Labels
samples = []
labels = []
for _, row in nprint_80.iterrows():
    # Check for no payload, all bits will be -1. There are more efficient ways to do this
    if len(set(row)) == 1:
        continue
    samples.append(np.array(row))
    labels.append('unencrypted')

for _, row in nprint_443.iterrows():
    # Check for no payload, all bits will be -1. There are more efficient ways to do this
    if len(set(row)) == 1:
        continue
    samples.append(np.array(row))
    labels.append('encrypted')
    
# Train and Test the Classifier
# Split data
X_train, X_test, y_train, y_test = train_test_split(samples, labels)
# Initialize Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)
# Train 
clf.fit(X_train, y_train) 
# Predict
y_pred = clf.predict(X_test)
# Statistics
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

   encrypted       0.67      1.00      0.80       440
 unencrypted       1.00      0.24      0.39       284

    accuracy                           0.70       724
   macro avg       0.84      0.62      0.60       724
weighted avg       0.80      0.70      0.64       724



## Conclusion

Hopefully this gives you a better idea of how nPrint can be used to rapidly train and test models for different traffic analysis problems. While this problem was contrived and simple, the same basic steps can be performed for any single-packet classification problem. If you want to train and test using **sets** of packets as input to a model, you'll either need a model that can handle that input, such as a CNN, or to flatten the 2D packet sample into a 1d sample for use with a model such as the random forest above.